# Full-circle Atomistic Comparison of ETS

## Experimental data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import facets_drivers

%store -r notebook_data
globals().update(notebook_data)

In [ ]:
from ewoksdask import execute_graph
if not local_run:
    from ewoksdask.schedulers import slurm_scheduler
    cluster = slurm_scheduler(account="staff-id01",queue="gpu", walltime="01:00:00", memory="40GB", cores=16, job_extra_directives=["--gres=gpu:1"])

In [ ]:
nodes = [
    {
        "id": "fc_analysis",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.FullCircleAnalysis",
    },
    {
        "id": "fc_analysis_sim",
        "task_type": "class",
        "task_identifier": "facets_drivers.facets.FullCircleAnalysis",
    }
]

workflow = {"graph": {"id": "FAC of ETS"}, "nodes": nodes}

inputs = [
    {"id": "fc_analysis", "name": "scratch_directory", "value": facets_directory},
    {"id": "fc_analysis", "name": "vti_filepath", "value": experimental_structure},
    {"id": "fc_analysis", "name": "hkl", "value": hkl},

    {"id": "fc_analysis_sim", "name": "scratch_directory", "value": facets_directory},
    {"id": "fc_analysis_sim", "name": "xyz_filepath", "value": f"{simulation_directory}/exp_minimized.xyz"},
    {"id": "fc_analysis_sim", "name": "hkl", "value": hkl},
]

In [ ]:
%%time
result = execute_graph(workflow, inputs=inputs, scheduler=None if local_run else cluster.scheduler_address)

In [ ]:
if not local_run:
    cluster.close()